# Programming Assingment #2

## Introduction 

The purpose of the assignment is to make ourselves familiarized with designing and programming algorithms for searching while it utilizes the most optimal paths to get to a specific goal using a heuristic function.

The goal for the assignment is to design and create the algorithms for two different environments given by the problems 9 and 11 [1].

Theoretical Background

The assignment uses an informed search methodology to address the problems. In informed search we utilize specific information of the problem to find more efficient solutions that may not be possible to obtain when we use uninformed search. To reach this goal, informed search algorithms make use of a heuristic function. A Heuristic function helps the search algorithm make a prediction about the cost of certain paths within the search. Heuristic functions are not the same for every problem; each problem requires a specific heuristic function that depends on the problem itself.

## Discussion

If the state space consists of all positions (x,y) on a plane, we can assume that the state space is infinite, since the positions of (x,y) in a plane are infinite. This means that the number of states within this space is infinite, thus making the number of paths to the goal infinite as well. With that being said, there is only one optimal path, this path being a straight line from the starting point to the goal. This is the ideal path if no obstacles are involved. Once obstacles are introduced to the problem, the solution still involves straight lines around said obstacles. For a space to be efficient, it should only include the vertices of the polygons, the start point and the goal. One basic concept of math we can apply to determine what would be the straight line from the start point to the goal, is the Pythagorean theorem, from which we can derive our heuristic function, this narrows the state space to only the space encompassed by the start point and goal or, for a more visual representation, the square formed by the starting point (x1, y1), the goal (x2, y2), the point (x2,y1), and the point (x1, y2). With this, the number of possible states and possible paths, becomes finite.

To solve the problem of identifying the shortest path between two points in a plane filled with polygon obstacle in python code we made use of the search.py [2] and utils.py [3] of the UC Berkeley code repository for python and our original python file of the problem mentioned above.

In [3]:
import math
from search import Problem, astar_search  


class PolygonPathProblem(Problem):
    def __init__(self, initial, goal, polygons, step_size=1):
        super().__init__(initial, goal)
        self.polygons = polygons
        self.step_size = step_size

    def actions(self, state):
        neighbors = self.get_neighbors(state)
        valid_actions = [neighbor for neighbor in neighbors if self.can_reach_directly(state, neighbor)]
        if self.can_reach_directly(state, self.goal):
            valid_actions.append(self.goal)
        return valid_actions

    def get_neighbors(self, state):
        step = self.step_size
        x, y = state
        # Generate neighboring nodes in a grid pattern
        return [(x+dx, y+dy) for dx in [-step, 0, step] for dy in [-step, 0, step] if not (dx == 0 and dy == 0)]

    def can_reach_directly(self, state, vertex):
        if state == vertex:
            return False
        for polygon in self.polygons:
            for i in range(len(polygon)):
                edge_start, edge_end = polygon[i], polygon[(i + 1) % len(polygon)]
                if self.do_lines_intersect(state, vertex, edge_start, edge_end):
                    return False
        return True

    @staticmethod
    def do_lines_intersect(line1_start, line1_end, line2_start, line2_end):
        def ccw(A, B, C):
            return (C[1] - A[1]) * (B[0] - A[0]) > (B[1] - A[1]) * (C[0] - A[0])

        A, B = line1_start, line1_end
        C, D = line2_start, line2_end

        return ccw(A, C, D) != ccw(B, C, D) and ccw(A, B, C) != ccw(A, B, D)


    def result(self, state, action):
        return action
 


    def h(self, node):
        return math.sqrt((node.state[0] - self.goal[0])**2 + (node.state[1] - self.goal[1])**2)


# Test cases
def test_cases():
    polygons = {
        "simple": [],
        "single_obstacle": [[[2, 2], [2, 3], [3, 3], [3, 2]]],
        "multiple_obstacles": [[[1, 2], [1, 3], [2, 3], [2, 2]], 
                               [[3, 1], [3, 2], [4, 2], [4, 1]]],
        "narrow_path": [[[1, 1], [1, 3], [2, 3], [2, 1]],
                        [[3, 1], [3, 3], [4, 3], [4, 1]]],
        "dead_end": [[[1, 1], [1, 2], [2, 2], [2, 1]], 
                     [[1, 3], [1, 4], [3, 4], [3, 3]]],
        "distant_goal": []
        } 

    start_goal_pairs = {
        "simple": ((0, 0), (5, 5)),
        "single_obstacle": ((0, 0), (4, 4)),
        "multiple_obstacles": ((0, 0), (5, 5)),
        "narrow_path": ((0, 0), (5, 5)),
        "dead_end": ((0, 0), (4, 0)),
        "distant_goal": ((0, 0), (10, 10))
    }

    problems = {name: PolygonPathProblem(start, goal, polygons[name])
                for name, (start, goal) in start_goal_pairs.items()}

    for name, problem in problems.items():
        solution = astar_search(problem, h = problem.h)
        print(f"Test Case '{name}': Path - {solution.solution() if solution else 'No path found'}, "
              f"Path Cost - {solution.path_cost if solution else 'N/A'}")


if __name__ == "__main__":
    test_cases()

Test Case 'simple': Path - [(5, 5)], Path Cost - 1
Test Case 'single_obstacle': Path - [(1, 1), (1, 2), (2, 3), (4, 4)], Path Cost - 4
Test Case 'multiple_obstacles': Path - [(1, 1), (1, 2), (2, 3), (3, 3), (5, 5)], Path Cost - 5
Test Case 'narrow_path': Path - [(1, 0), (1, 1), (2, 2), (2, 3), (3, 3), (5, 5)], Path Cost - 6
Test Case 'dead_end': Path - [(4, 0)], Path Cost - 1
Test Case 'distant_goal': Path - [(10, 10)], Path Cost - 1


For the output, the bigger the Path Cost, the more deviations from the straight line were made. Let's take the simple, which has no obstacles, and the multiple_obstacles as examples. The simple test case has a Path Value of 1, which means the the algorithm followed a straight line from the start point to the goal, while the multiple_obstacles problem has a Path Cost of 6, meaning that the algorithm made 5 aditional steps to the amount expected if it could travel in a straight line.

The next exercise is the missionaries and cannibals problem. The problem states that there are six people on one side of a river, three missionaries and three cannibals alongside a boat that can hold one or two people at a time. Every person has to cross to the other side of the river using the boat, however the missionaries must not be outnumbered by cannibals on either side at all times.

The initial state of the problem is three missionaries, three cannibals, and the boat are on the left side of a river. The goal state is that all three missionaries and cannibals are on the right side of the river. The operator allowed is that the boat can carry one or two people from one side to the other, the boat cannot travel without a person on it. And as constraint we have that at no point should the number of cannibals on either side of the river outnumber the number of missionaries, if it happens the cannibals will eat the missionaries and the problem will fail. The problem is solved using Breadth-First-Tree-Search (BFTS) because it guarantees the shortest path to the goal state while avoiding any repeated states. In the code it checks the validity of states, generates the next possible states and maintains a queue to explore the state space. The importance of checking for repeated states is crucial in the BFTS to avoid infinite loops, as the problem space can lead to revisiting the same states. The process is showcased in Figure 3 which is the output of the code that was implemented for the problem


In [1]:
import search

""" Missionaries and Cannibals problem"""

# Defining initial states
initial_state = (3, 3, 1) # (Missionaries on left, Cannibals on left, Boat position 1 for left or 0 for right)
goal_state = (0, 0, 0)

# Function to check if state is valid
def is_valid(state):
    miss, canni, boat = state
    if miss < 0 or canni < 0 or miss > 3 or canni > 3 or (canni > miss > 0) or (3 - canni > 3 - miss > 0):
        return False
    return True

# Define a function to generate next possible states
def generate_next_states(state):
    miss, canni, boat = state
    possible_states = []

    # Define the possible moves
    moves = [(2, 0), (0, 2), (1, 1), (1, 0), (0, 1)]

    for move in moves:
        if boat == 1:
            new_state = (miss - move[0], canni - move[1], 0)
        else:
            new_state = (miss + move[0], canni + move[1], 1)
        if is_valid(new_state):
            possible_states.append(new_state)
    return possible_states

# Define the Problem class
class MissionariesCannibalsProblem:
    def __init__(self, initial_state, goal_state):
        self.initial = initial_state
        self.goal = goal_state

    def actions(self, state):
        return generate_next_states(state)

    def result(self, state, action):
        return action

    def goal_test(self, state):
        return state == self.goal

    def path_cost(self, c, state1, action, state2):
        return 1
#  Instantiate the problem
problem = MissionariesCannibalsProblem(initial_state, goal_state)

# Apply the problem in the breadth_first_tree
solution_node = search.breadth_first_tree_search(problem)

# Print the solution
if solution_node:
    solution_path = []
    while solution_node:
        solution_path.append(solution_node.state)
        solution_node = solution_node.parent
    solution_path.reverse()
    for step, state in enumerate(solution_path):
        print(f"Step {step + 1}: {state}")
else:
    print("No solution found.")

Step 1: (3, 3, 1)
Step 2: (3, 1, 0)
Step 3: (3, 2, 1)
Step 4: (3, 0, 0)
Step 5: (3, 1, 1)
Step 6: (1, 1, 0)
Step 7: (2, 2, 1)
Step 8: (0, 2, 0)
Step 9: (0, 3, 1)
Step 10: (0, 1, 0)
Step 11: (1, 1, 1)
Step 12: (0, 0, 0)


This puzzle can be challenging to people because it involves multiple constraints at the same time like maintaining the safety of missionaries, preventing the cannibals from outnumbering missionaries on either side and efficiently using the boats capacity. Although it is easy to represent the state space, solving the problem manually is not simple due to its constraints and requirement for a valid solution. This problem requires careful planning to make sure that it doesn't go against any restrictions when finding a solution

## Conclusion

In conclusion, tackling the two proposed problems, key learnings encompass the realms of problem-solving and algorithmic design, with a specific focus on graph theory, heuristics, state space analysis, and optimization strategies. These problems helped us understand how to represent problems using graphs, the development and application of heuristic functions, and the importance of understanding constraints and safety conditions.  Additionally, they offer insights into human cognitive processes, highlighting the distinction between human and algorithmic problem-solving approaches. This also serves as a practical application of AI techniques and reinforces the importance of algorithm performance evaluation, providing a comprehensive understanding of both the theoretical and applied aspects of artificial intelligence and computational problem-solving.

## References

References:

Exercise 9 and 11 https://aimacode.github.io/aima-exercises/search-exercises/

search.py 
https://github.com/aimacode/aima-python/blob/master/search.py

utils.py
https://github.com/aimacode/aima-python/blob/master/utils.py
